In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdMolDescriptors, QED


In [3]:


def fetch_admet(input_csv, output_csv, smiles_col="smiles", name_col="drug_name"):
    df = pd.read_csv(input_csv)
    results = []
    
    for _, row in df.iterrows():
        smi = row[smiles_col]
        mol = Chem.MolFromSmiles(smi) if pd.notna(smi) else None
        
        if mol is None:
            results.append({smiles_col: smi, "error": "Invalid SMILES"})
            continue
        
        results.append({
            smiles_col: smi,
            "MW": round(Descriptors.MolWt(mol), 2),
            "LogP": round(Descriptors.MolLogP(mol), 2),
            "HBD": Lipinski.NumHDonors(mol),
            "HBA": Lipinski.NumHAcceptors(mol),
            "TPSA": round(Descriptors.TPSA(mol), 2),
            "RotatableBonds": Lipinski.NumRotatableBonds(mol),
            "HeavyAtoms": Lipinski.HeavyAtomCount(mol),
            "NumRings": rdMolDescriptors.CalcNumRings(mol),
            "NumAromaticRings": rdMolDescriptors.CalcNumAromaticRings(mol),
            "FractionCSP3": round(rdMolDescriptors.CalcFractionCSP3(mol), 3),
            "QED": round(QED.qed(mol), 3),
            "MolarRefractivity": round(Descriptors.MolMR(mol), 2),
            "Lipinski_Violations": sum([
                Descriptors.MolWt(mol) > 500,
                Descriptors.MolLogP(mol) > 5,
                Lipinski.NumHDonors(mol) > 5,
                Lipinski.NumHAcceptors(mol) > 10
            ]),
            "Veber_Violations": sum([
                Lipinski.NumRotatableBonds(mol) > 10,
                Descriptors.TPSA(mol) > 140
            ]),
        })
    
    results_df = pd.DataFrame(results)
    if name_col in df.columns:
        results_df.insert(0, name_col, df[name_col].values[:len(results_df)])
    
    results_df.to_csv(output_csv, index=False)
    print(f"Saved {len(results_df)} results to {output_csv}")
    return results_df

if __name__ == "__main__":
    fetch_admet("ddms1.csv", "admet_results.csv")

Saved 149 results to admet_results.csv
